# Python에서 변수의 메모리 구조와 참조값 크기

Python에서 변수의 메모리 사용량을 정확히 이해해보겠습니다.

## 1. 변수 `a`의 실제 크기

In [4]:
import sys

a = [1, 2, 3]
print(f"변수 a가 참조하는 객체의 크기: {sys.getsizeof(a)} bytes")
print(f"정수 1의 크기: {sys.getsizeof(1)} bytes")
print(f"정수 2의 크기: {sys.getsizeof(2)} bytes")
print(f"정수 3의 크기: {sys.getsizeof(3)} bytes")

변수 a가 참조하는 객체의 크기: 88 bytes
정수 1의 크기: 28 bytes
정수 2의 크기: 28 bytes
정수 3의 크기: 28 bytes


## 2. 참조값(포인터)의 크기

Python에서 **변수 자체**는 실제로 참조값(포인터)을 저장하지만, 그 크기는 시스템에 따라 다릅니다:

- **32비트 시스템**: 포인터 크기는 4바이트 (32비트)
- **64비트 시스템**: 포인터 크기는 8바이트 (64비트)

In [5]:
import sys

# 현재 시스템의 포인터 크기 확인
print(f"현재 시스템의 포인터 크기: {sys.getsizeof(0)} bytes")
print(f"플랫폼 정보: {sys.maxsize > 2**32}")  # True면 64비트

현재 시스템의 포인터 크기: 28 bytes
플랫폼 정보: True


## 3. 실제 메모리 구조

``` python
a = [1, 2, 3]
```

이 코드에서 메모리 구조는 다음과 같습니다:

``` text
스택 영역:
┌─────────────────┐
│ 변수 a          │ ← 8바이트 (64비트 시스템에서 참조값)
│ 0x7f8b8c0d1e40  │
└─────────────────┘
        │
        ▼
힙 영역:
┌─────────────────┐
│ 리스트 객체     │ ← 실제 리스트 객체 (훨씬 큰 크기)
│        │
│ - 타입 정보     │
│ - 크기 정보     │
│ - 요소들의 참조 │
└─────────────────┘
```

## 4. 정확한 크기 측정

In [6]:
import sys
import ctypes

a = [1, 2, 3]

# 리스트 객체 자체의 크기
print(f"리스트 객체의 크기: {sys.getsizeof(a)} bytes")

# 참조값(포인터)의 크기는 시스템 의존적
pointer_size = ctypes.sizeof(ctypes.c_void_p)
print(f"포인터 크기: {pointer_size} bytes")

# 64비트 시스템에서는 8바이트, 32비트에서는 4바이트
print(f"현재 시스템: {'64비트' if pointer_size == 8 else '32비트'}")

리스트 객체의 크기: 88 bytes
포인터 크기: 8 bytes
현재 시스템: 64비트


## 5. 결론

**변수 `a` 자체**는 참조값을 저장하므로:
- **32비트 시스템**: 4바이트
- **64비트 시스템**: 8바이트

하지만 `a`가 **참조하는 리스트 객체**는 훨씬 큽니다:
- 리스트 객체의 메타데이터
- 각 요소에 대한 참조값들
- 동적 배열을 위한 추가 공간

## 6. 실제 확인 코드

In [7]:
import sys

# 다양한 객체들의 크기 비교
objects = [
    ([1, 2, 3], "리스트 [1,2,3]"),
    ([], "빈 리스트"),
    (1, "정수 1"),
    ("hello", "문자열 'hello'"),
    ({}, "빈 딕셔너리")
]

for obj, desc in objects:
    print(f"{desc}: {sys.getsizeof(obj)} bytes")

# 포인터 크기 확인
import ctypes
print(f"\n포인터 크기: {ctypes.sizeof(ctypes.c_void_p)} bytes")

리스트 [1,2,3]: 88 bytes
빈 리스트: 56 bytes
정수 1: 28 bytes
문자열 'hello': 46 bytes
빈 딕셔너리: 64 bytes

포인터 크기: 8 bytes


## 최종 답변

따라서 질문에 대한 답은: **현대의 64비트 시스템에서는 변수 `a`가 저장하는 참조값은 8바이트**이고, **32비트 시스템에서만 4바이트**입니다.

### 추가 실험: 메모리 주소 확인

In [8]:
# 메모리 주소 확인
a = [1, 2, 3]
b = [1, 2, 3]
c = a

print(f"a의 메모리 주소: {id(a)}")
print(f"b의 메모리 주소: {id(b)}")
print(f"c의 메모리 주소: {id(c)}")
print(f"\na와 c는 같은 객체를 참조: {a is c}")
print(f"a와 b는 다른 객체를 참조: {a is b}")

# 16진수로 주소 표시
print(f"\na의 주소 (16진수): 0x{id(a):x}")
print(f"b의 주소 (16진수): 0x{id(b):x}")

a의 메모리 주소: 2178403050560
b의 메모리 주소: 2178403049472
c의 메모리 주소: 2178403050560

a와 c는 같은 객체를 참조: True
a와 b는 다른 객체를 참조: False

a의 주소 (16진수): 0x1fb32f0a440
b의 주소 (16진수): 0x1fb32f0a000
